# Testando API do Trello

In [1]:
import requests
import json
import pandas as pd


In [ ]:
#Token para teste
#token_trello = '8b87e84eab71efca714b5c58277867922cd63a26087236eb3bc5d8db0e82dd28'

#Key para teste
#key_trello = 'fc23aaf60dd87002c17a29e2fc039d19'

#Endpoint para requests
#main_trello_endpoint = 'https://api.trello.com/1/'


## GET Request

### Getting jsonObj from board by id

In [ ]:
#def getBoard(id):

#    url = main_trello_endpoint+'boards/'+id
#    params = {'key': key_trello, 'token': token_trello}
#    response = requests.get(url=url, params=params)
#    boardData = response.json()
#    print(boardData)


#getBoard('C65vXFhE')


## Creating Board Class

In [7]:
class Board:
    def __init__(self, key='fc23aaf60dd87002c17a29e2fc039d19', token='8b87e84eab71efca714b5c58277867922cd63a26087236eb3bc5d8db0e82dd28', main_endpoint='https://api.trello.com/1/', boardData = None, boardID = input(), boardLists = None) -> None:
        self.key = key
        self.token = token
        self.main_endpoint = main_endpoint
        self.boardId = boardID
        self.boardData = boardData
        self.boardLists = boardLists
    
    def get_board(self):
        #Pega url base e acrescenta a parte do link faltante nesse caso "boards"
        url = self.main_endpoint +'boards/'+self.boardId
        params = {'key': self.key, 'token': self.token}
        response = requests.get(url=url, params=params)
        #Salva o jsonObj na variável boardData em forma de dicionário
        self.boardData = response.json()
        return self.boardData

    def get_board_name(self):
        #Verifica se as informações já foram puxadas do Trello
        if self.boardData == None:
            self.get_board()

        boardName = self.boardData['name']
        return(boardName)

    def get_board_id(self):
        #Verifica se as informações já foram puxadas do Trello
        if self.boardData == None:
            self.get_board()
        
        #pega o id do quadro
        boardIdLocal = self.boardData['id']

        return boardIdLocal

    def get_board_lists(self):
        #Acrescenta palavra ao link '/lists' 
        url = self.main_endpoint +'boards/'+self.boardId+'/lists'
        params = {'key': self.key, 'token': self.token}
        response = requests.get(url=url, params=params)
        self.boardLists = response.json()
        
        return self.boardLists

    def get_board_lists_names_and_idLists(self):
        self.get_board_lists()
        #Com certeza tem método mais eficiente -----------------------------------------------------------revisitar

        #adiciona nomes a lista nameList
        numberOfNames = len(self.boardLists)
        holder = None
        nameList = []  
        for i in range(numberOfNames):
            holder = dict(self.boardLists[i])
            nameList.append(holder['name'])

        #adiciona ids a lista idListList
        holder = None
        idListList = []
        for i in range(numberOfNames):
            holder = dict(self.boardLists[i])
            idListList.append(holder['id'])    
        
        #concatena nomes e ids no dicionario name_idList
        name_idList = dict(zip(nameList, idListList))

        return name_idList
        
    def get_idCards_on_board(self):
        self.get_board_lists_names_and_idLists()
        self.get_board_id()
        #altera link '/lists' para '/cards'
        url = self.main_endpoint +'boards/'+self.boardId+'/cards'
        params = {'key': self.key, 'token': self.token}
        response = requests.get(url=url, params=params)
        cards = response.json()
        
        #Faz uma lista com todos os ids dos cartoes
        numberOfCards = len(cards)
        holder = None
        cardsOnBoardIds = []
        for i in range(numberOfCards):
            holder = dict(cards[i])
            cardsOnBoardIds.append(holder['id'])  

         #Faz uma lista com todos os ids da lista que o cartao esta
        numberOfCards = len(cards)
        holder = None
        cardsOnBoardIdList = []
        for i in range(numberOfCards):
            holder = dict(cards[i])
            cardsOnBoardIdList.append(holder['idList'])
        
        #dicionario no formato {IdCartoes: IdLista que o cartao esta}
        CardsOnBoard_CardsOnBoardIdList = dict(zip(cardsOnBoardIds, cardsOnBoardIdList))

        return CardsOnBoard_CardsOnBoardIdList

    def get_a_card():
        pass

    def create_card_on_backlog(self):
        lists_and_ids = self.get_board_lists_names_and_idLists()
        key_holder = lists_and_ids['Backlog de Vídeos']


#board id -> C65vXFhE
teste = Board()
#teste.get_board_name()
#teste.get_board_id()
#teste.get_board()
#teste.get_board_lists_names_and_idLists()
#teste.get_board_id()
#teste.get_cards_on_board()
#teste.get_board_lists()
teste.create_card()


'63735fafa91e2905e08ca86a'

# Create Card